In [ ]:
# !earthengine authenticate

# NOTE: instead of running this in the notebook use the binder terminal
# 1. go to File > new > terminal
# 2. enter `earthengine authenticate` & follow prompts

In [1]:
cd /home/jovyan

/home/jovyan


In [ ]:
# check to ensure the file we need is there
!ls -lh '/home/jovyan/.config/earthengine/credentials'

#### Import EE

In [25]:
import ee
import geemap
import pandas as pd
ee.Initialize()

In [2]:
## Verify the ee module is loaded.
print(ee.__version__)

0.1.286


#### Import asset collections

Ocean Color and SST4 daily product (2002-Present) from MODIS at 1 km per pixel (FLORIDA).

OC Bands:\
b1 = Rrs 412, b2 = Rrs 443, b3 = Rrs 488, b4 = Rrs 531, b5 = Rrs 547, b6 = Rrs 667, b7 = Rrs 678, b8 = chlor a (mg/m3), b9 = nflh

SST4 Bands:\
b1 = SST (°C)

In [3]:
## Florida MODIS L3 data (uploaded from imars server)
oc = ee.ImageCollection('projects/imars-3d-wetlands/MODIS_L3/FL_OC') ## Ocean Color
sst = ee.ImageCollection('projects/imars-3d-wetlands/MODIS_L3/FL_SST4') ## SST

#### Monthly means

Define some variables first

In [18]:
band = 'b1' #Type same band in Line 149
renameBand = 'SST'
collection = sst
start_date = '2003-01-01'
end_date = '2020-12-31'

# Establish lists with months and years of selected time range above.
months = ee.List.sequence(1, 12);
years = ee.List.sequence(2003, 2020);

# Establish lists with months and years of selected time range above.
months = ee.List.sequence(1, 12);
years = ee.List.sequence(2003, 2020);

## lat/lon limits
region = ee.Geometry.Polygon([[[-86.25, 22.75], [-78.65, 22.75], [-78.65, 31.15], [-86.25, 31.15], [-86.25, 22.75]]])

## Visualization parameters
palette = ['030d81','0519ff','05e8ff','11ff01','fbff01','ff9901','ff0000','ad0000']
viz = {'min':25,'max':31,'palette':palette}

Filter collection and get monthly means:

In [8]:
## Filter collection
filtered = collection.filterDate(ee.Date(start_date), ee.Date(end_date)).select(band)

## Get monthly menas in the defined time range
byMonthYear = ee.ImageCollection.fromImages(
  years.map(lambda y: 
            months.map(lambda m:
                              filtered\
                              .filter(ee.Filter.calendarRange(y, y, 'year'))\
                              .filter(ee.Filter.calendarRange(m, m, 'month'))\
                              .reduce(ee.Reducer.mean())\
                              .set('month', m).set('year', y))).flatten())

print('Monthly Means:',byMonthYear.size().getInfo())

Monthly Means: 216


#### Visualize the first image from the collection as example

In [ ]:
# initialize our map
Map = geemap.Map()
Map.centerObject(region, 6)
Map.addLayer(byMonthYear.first(), viz, "Florida")

Map.addLayerControl()
Map

#Map.addLayer(byMonthYear.first(),vis_chlor,'image')
#

#### Get mean values from each image

In [20]:
## Function to get mean values
def getStats(img):
    ## Select and rename bands.
    #image = ee.Image(img).select(band+'_mean').rename(renameBand)
    image = ee.Image(img).rename(renameBand)
    ## Create and combine reducers of mean and std.
    reducers = ee.Reducer.mean().combine(
        reducer2= ee.Reducer.stdDev(),
        sharedInputs= True)
    ## Apply reducers to the image.
    stats = image.reduceRegion(
        reducer= reducers,
        geometry= region,
        #bestEffort= True,
        scale= 1000,
        maxPixels= 1e9)
    
    ## Create feature variable to allocate data.
    feature = ee.Feature(None)
    ## Set properties of interest.
    properties = ['system:index','month','year',renameBand+'_mean',renameBand+'_stdDev']
    ## Apply the stats on every image and collect the output.
    imageStats = image.setMulti(stats)
    ## return feature with the selected properties.
    return ee.Feature(feature).copyProperties(imageStats, properties)

In [21]:
stats = byMonthYear.map(getStats)

#### Get data in pandas format

In [27]:
mean = pd.DataFrame(stats.aggregate_array('SST_mean').getInfo(), columns=['Mean'])
month = pd.DataFrame(stats.aggregate_array('month').getInfo(), columns=['Month'])
year = pd.DataFrame(stats.aggregate_array('year').getInfo(), columns=['Year'])

df = pd.concat([year, month, mean], axis=1)#.rename(index=rowIndex)
print(df)

     Year  Month       Mean
0    2003      1  22.150675
1    2003      2  22.137196
2    2003      3  24.294582
3    2003      4  23.891055
4    2003      5  26.836783
..    ...    ...        ...
211  2020      8  29.699092
212  2020      9  29.201843
213  2020     10  28.101500
214  2020     11  26.275768
215  2020     12  23.826473

[216 rows x 3 columns]


#### Save file in locati